### 此程序用于《毕业论文》3.1榆中站气象要素的季节变化特征分析
---
### 前期数据处理流程
1. 原始数据为http://weather.uwyo.edu/upperair/seasia.html 获得的FSL文件
2. 经过read_file_day.pro处理后得到每次探测数据的分离文件，得到FSL（文件夹）数据
3. 经过Output_Yuzhong_2010_2018_As_MWMOD_Input_Format.pro处理后，转换成相对湿度并按照高度给定高度插值，得到FSL1（文件夹）数据
### 本程序完成的后续工作
4. 从FSL1数据中提取某些气象要素的季节变化信息
5. 绘制垂直高度剖面图
### 关键步骤
- 异常值处理：删除该观测时间点数据
- 注意修改取样日期、时间、横轴坐标宽度

In [1]:
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import calendar

In [8]:
#@ 数据搜索
target_str = ['2009']
Ti = '12'
dirpath = r'I:\Data\Personal Data\graduation project\yuzhongzhan\FSL1'
filepaths = []
for root,dirs,files in os.walk(dirpath):
    for file in files:
        if((len(target_str[0])==4)and(str(file[30:34]) in target_str)and(file[38:40]==Ti)):
            filepaths.append(os.path.join(root,file))
        elif((len(target_str[0])==6)and(file[30:36] in target_str)and(file[38:40]==Ti)):
            filepaths.append(os.path.join(root,file))
print(len(filepaths))
for filepath in filepaths:
    print(filepath)

364
I:\Data\Personal Data\graduation project\yuzhongzhan\FSL1\Yuzhong_As_MWMOD_Input_Format_2009010112.dat
I:\Data\Personal Data\graduation project\yuzhongzhan\FSL1\Yuzhong_As_MWMOD_Input_Format_2009010212.dat
I:\Data\Personal Data\graduation project\yuzhongzhan\FSL1\Yuzhong_As_MWMOD_Input_Format_2009010312.dat
I:\Data\Personal Data\graduation project\yuzhongzhan\FSL1\Yuzhong_As_MWMOD_Input_Format_2009010412.dat
I:\Data\Personal Data\graduation project\yuzhongzhan\FSL1\Yuzhong_As_MWMOD_Input_Format_2009010512.dat
I:\Data\Personal Data\graduation project\yuzhongzhan\FSL1\Yuzhong_As_MWMOD_Input_Format_2009010612.dat
I:\Data\Personal Data\graduation project\yuzhongzhan\FSL1\Yuzhong_As_MWMOD_Input_Format_2009010712.dat
I:\Data\Personal Data\graduation project\yuzhongzhan\FSL1\Yuzhong_As_MWMOD_Input_Format_2009010812.dat
I:\Data\Personal Data\graduation project\yuzhongzhan\FSL1\Yuzhong_As_MWMOD_Input_Format_2009010912.dat
I:\Data\Personal Data\graduation project\yuzhongzhan\FSL1\Yuzhong_As_

In [9]:
#@ 数据预处理
drawing_data = pd.DataFrame()
#@ 选择要处理的气象信息
head_col = ['Press','Altitude','Temperature','Relative_humidity','NaN1','NaN2','NaN3']
Meteorological_elements = head_col[3]
print('\n'+Meteorological_elements)
#@ 数据框架
data = pd.read_table(filepaths[0],sep='\s+',skiprows=6,names=head_col)
drawing_data['Altitude'] = data['Altitude']
for i in range(len(filepaths)-1):
    data = pd.read_table(filepaths[i],sep='\s+',skiprows=6,names=head_col)
    drawing_data[filepaths[i][88:96]] = data[Meteorological_elements]
    # if(data.mean()[Meteorological_elements] >= 0): #@ 删除异常值
    #     drawing_data[filepaths[i][88:98]] = data[Meteorological_elements]
#@ 异常值替换为NaN
drawing_data[drawing_data < 0] = np.nan

if(len(target_str)==6):
    print(f'理想样本数量为{calendar.monthrange(int(target_str[0:4]),int(target_str[4:6]))[1]}')
else:
    nums = 0
    for ta in target_str:
        for i in range(12):
            nums += calendar.monthrange(int(ta),i+1)[1]
    print(f'\n理想样本数量为{nums}')

print(f'搜集到样本共计{drawing_data.shape[1]-1}')


Relative_humidity

理想样本数量为365
搜集到样本共计363


In [10]:
drawing_data.to_csv(r'I:\Data\Personal Data\graduation project\yuzhongzhan'+'\\'+Meteorological_elements+target_str[0]+'_Ti'+Ti+ '.csv')

In [11]:
drawing_data = drawing_data.iloc[:,1:]

In [12]:
#@ 剖面图绘制
fig = plt.figure(figsize=(5,4),dpi=800,frameon=False) #@新建画布
ax = fig.add_axes([0,0,4,2],facecolor='gray')
#@ 坐标轴标签
ax.set_xlabel('Date',fontsize=20)
ax.set_ylabel('Altitude km',fontsize=20)

if(Meteorological_elements==head_col[3]):
    level = np.arange(0,110,10)
elif(Meteorological_elements==head_col[2]):
    level = np.arange(200,300,10)
ca = ax.contourf(drawing_data,10,cmap='jet',levels=level)
# cb = ax.contour(drawing_data,colors='black',levels=level)
#@ 图例
cbar = plt.colorbar(ca)
cbar.set_label(Meteorological_elements+' K',fontsize=20)
# cbar.set_ticks([level]) 
#@ X轴坐标 获取每个月第一天和全年最后一天作为横轴坐标
month = 00 # flag标志着找到了每个月的第一天
x_ticks = []
x_labels = []
for i in range(drawing_data.shape[1]):
    if(month != drawing_data.columns[i][0:6]):
        month = drawing_data.columns[i][0:6]
        x_ticks.append(i)
        x_labels.append(drawing_data.columns[i][4:8])
x_ticks.append(drawing_data.shape[1]-1)
x_labels.append(drawing_data.columns[drawing_data.shape[1]-1][4:8])
# x_ticks = np.arange(0,drawing_data.shape[1],50)
# x_ticks = np.append(x_ticks,drawing_data.shape[1]-1)
# x_labels = []
# for i in range(len(x_ticks)):
#     x_labels.append(drawing_data.columns[x_ticks[i]][4:8])
print(x_ticks,x_labels)
plt.xticks(x_ticks,x_labels,rotation=45,fontsize=20)
#@ Y轴坐标
y_ticks = [0,2,4,6,8,10,14,18,22,26,30,34,38,42,46]
y_labels = ['0','0.2','0.4','0.6','0.8','1.0','2.0','3.0','4.0','5.0','6.0','7.0','8.0','9.0','10.0']
plt.yticks(y_ticks,y_labels,fontsize=20)

#@ 储存图片
picture_path = r'I:\Data\Personal Data\graduation project\yuzhongzhan\picture'+'\\'+Meteorological_elements+target_str[0]+'_'+'Ti'+Ti+'.svg'
plt.savefig(picture_path,bbox_inches = 'tight')
# #@ 展示图片
plt.show()

[0, 31, 59, 90, 120, 151, 181, 212, 242, 272, 303, 333, 362] ['0101', '0201', '0301', '0401', '0501', '0601', '0701', '0801', '0901', '1001', '1101', '1201', '1230']
